<a href="https://colab.research.google.com/github/cbonnin88/VifStream/blob/main/A_BTesting_VifStream.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import auth
from google.cloud import bigquery
import polars as pl
from scipy import stats
import plotly.express as px
import plotly.graph_objects as go

In [2]:
auth.authenticate_user()
print('Authentication Successfull')

Authentication Successfull


In [3]:
project_id = 'poetic-maxim-486609-e5'
client = bigquery.Client(project=project_id)

In [4]:
query = """
SELECT * FROM `poetic-maxim-486609-e5.vifstream_dbt.fct_user_retention`
"""
df_pandas_temp = client.query(query).to_dataframe()
df_fct = pl.from_pandas(df_pandas_temp)

print(f'Successfully pulled {df_fct.height} rows from BigQuery')
display(df_fct.head())

Successfully pulled 1000000 rows from BigQuery


event_id,user_id,event_name,event_at,signup_date,plan_type,device,days_since_signup
i64,i64,str,"datetime[μs, UTC]",date,str,str,i64
119801,17807,"""add_to_favorites""",2025-12-10 05:15:00.625466 UTC,2024-03-31,"""free""","""Mobile""",619
134920,19389,"""add_to_favorites""",2025-12-04 14:07:04.038319 UTC,2025-01-25,"""free""","""Mobile""",313
592737,41747,"""add_to_favorites""",2025-09-19 15:04:54.974391 UTC,2025-04-02,"""free""","""Mobile""",170
359413,3974,"""add_to_favorites""",2025-12-26 20:16:38.554958 UTC,2025-10-10,"""free""","""Mobile""",77
402682,35146,"""add_to_favorites""",2026-01-05 03:26:35.243159 UTC,2026-01-18,"""premium""","""Mobile""",-13


# **Simulating an A/B Testing Assignment**

In [5]:
# Even ID's = Control, Odd ID's = Variant

df_ab_test = df_fct.with_columns(
    pl.when(pl.col('user_id') % 2 ==0)
    .then(pl.lit('Group A (Control)'))
    .otherwise(pl.lit('Group B (Variant)'))
    .alias('ab_group')
)

display(df_ab_test.head())

event_id,user_id,event_name,event_at,signup_date,plan_type,device,days_since_signup,ab_group
i64,i64,str,"datetime[μs, UTC]",date,str,str,i64,str
119801,17807,"""add_to_favorites""",2025-12-10 05:15:00.625466 UTC,2024-03-31,"""free""","""Mobile""",619,"""Group B (Variant)"""
134920,19389,"""add_to_favorites""",2025-12-04 14:07:04.038319 UTC,2025-01-25,"""free""","""Mobile""",313,"""Group B (Variant)"""
592737,41747,"""add_to_favorites""",2025-09-19 15:04:54.974391 UTC,2025-04-02,"""free""","""Mobile""",170,"""Group B (Variant)"""
359413,3974,"""add_to_favorites""",2025-12-26 20:16:38.554958 UTC,2025-10-10,"""free""","""Mobile""",77,"""Group A (Control)"""
402682,35146,"""add_to_favorites""",2026-01-05 03:26:35.243159 UTC,2026-01-18,"""premium""","""Mobile""",-13,"""Group A (Control)"""


# **Aggregating the Metric**

In [7]:
ab_summary = df_ab_test.group_by('user_id').agg(
    pl.col('ab_group').first().alias('ab_group'),
    # Creating a Boolean: 1 if they tuned in at least once, else 0
    pl.col('event_name').filter(pl.col('event_name') == 'channel_tune_in').count().gt(0).cast(pl.Int32).alias('converted')
)

results = ab_summary.group_by('ab_group').agg(
    pl.col('converted').count().alias('total_users'),
    pl.col('converted').sum().alias('conversions'),
    pl.col('converted').mean().alias('conversion_rate')
)

display(results.head())

ab_group,total_users,conversions,conversion_rate
str,u32,i32,f64
"""Group B (Variant)""",25000,24997,0.99988
"""Group A (Control)""",25000,25000,1.0


# **Statistical Significance (The Chi-Squared Test)**

- Since conversion is a 'Yes/No' metric, I used the Chi-Squared test (or a Z-test) rather than a T-test.

In [8]:
conversions = results.sort('ab_group')['conversions'].to_list()
total_users = results.sort('ab_group')['total_users'].to_list()

In [11]:
# Chi-squared contingency table
# [[converted_A, not_converted_A], [converted_B, not_converted_B]]

contingency_table = [
    [conversions[0], total_users[0] - conversions[0]],
    [conversions[1], total_users[1] - conversions[1]]
]

chi2,p_value, _, _ = stats.chi2_contingency(contingency_table)
print(f'P-value: {p_value:.4f}')
if p_value < 0.05:
  print('✨ Statistically Significant! The difference is real.')
else:
  print('🤷 Not Significant. The difference is likely due to noise.')

P-value: 0.2482
🤷 Not Significant. The difference is likely due to noise.


# **Confidence Interval**

In [16]:
fig_confidence = px.bar(
    results.to_pandas(),
    x='ab_group',
    y='conversion_rate',
    color='ab_group',
    title='A/B Test Results: Conversion to Tune-In',
    labels={'conversion_rate':'Conversion Rate (%)'},
    text_auto='.2%'
)

fig_confidence.update_layout(showlegend=False,template='plotly_white')
fig_confidence.show()